In [1]:
import pandas as pd
import plotly.graph_objects as go
import base64
import imageio.v3 as iio
import numpy as np
import os

# Load the data
data = pd.read_csv("superbowl.csv")

# Clean and process the data
data['Winner'] = data['Winner'].str.strip()
data['Year'] = pd.to_datetime(data['Date']).dt.year

# Aggregate wins by year and winner
wins = data.groupby(['Year', 'Winner']).size().reset_index(name='Wins')

# Get all unique teams and years
teams = wins['Winner'].unique()
years = range(wins['Year'].min(), wins['Year'].max() + 1)

# Create a complete grid for all teams and years
all_data = pd.DataFrame([(team, year) for team in teams for year in years], columns=['Winner', 'Year'])

# Merge with actual wins data
final_df = pd.merge(all_data, wins, on=['Winner', 'Year'], how='left').fillna(0)

# Add a placeholder year with 0 cumulative wins for all teams
placeholder_year = final_df['Year'].min() - 1
placeholders = pd.DataFrame({'Winner': teams, 'Year': placeholder_year, 'Wins': 0})

# Append placeholders to the final DataFrame
final_df = pd.concat([placeholders, final_df], ignore_index=True)

# Calculate cumulative wins
final_df['Cumulative Wins'] = final_df.groupby('Winner', observed=False)['Wins'].cumsum()

# Force all teams to appear on the y-axis for each frame
final_df['Winner'] = pd.Categorical(final_df['Winner'], categories=teams, ordered=True)

# Mapping from team names to logo filenames
team_logo_mapping = {
    'Packers': 'GB.png',
    'Jets': 'NYJ.png',
    'Chiefs': 'KC.png',
    'Colts': 'IND.png',
    'Cowboys': 'DAL.png',
    'Dolphins': 'MIA.png',
    'Steelers': 'PIT.png',
    'Raiders': 'OAK.png',
    '49ers': 'SF.png',
    'Commanders': 'WASNEW.png',
    'Bears': 'CHI.png',
    'Giants': 'NYG.png',
    'Broncos': 'DEN.png',
    'Rams': 'LA.png',
    'Ravens': 'BAL.png',
    'Patriots': 'NE.png',
    'Buccaneers': 'TB.png',
    'Saints': 'NO.png',
    'Seahawks': 'SEA.png',
    'Eagles': 'PHI.png'
}

# Manually assign specific colors to each team
team_colors = {
    'Packers': '#1e8100',    # Green
    'Jets': '#1d6e30',       # Dark Green
    'Chiefs': '#e8000e',     # Red
    'Colts': '#003d73',      # Blue
    'Cowboys': '#0060A1',    # Blue
    'Dolphins': '#008E97',   # Teal
    'Steelers': '#FFB81C',   # Gold
    'Raiders': '#000000',    # Black
    '49ers': '#D50032',      # Red
    'Commanders':'#D50032',  # Red
    'Bears': '#0B162A',      # Navy Blue
    'Giants': '#0A2261',     # Blue
    'Broncos': '#002244',    # Navy
    'Rams': '#003B5C',       # Blue
    'Ravens': '#241773',     # Purple
    'Patriots': '#003B5C',   # Navy Blue
    'Buccaneers': '#D50A0A', # Red
    'Saints': '#D3BC8D',     # Gold
    'Seahawks': '#006747',   # Green
    'Eagles': '#004C54'      # Dark Green
}

# Function to encode images to Base64
def image_to_base64(image_path):
    try:
        with open(image_path, "rb") as img_file:
            return f"data:image/png;base64,{base64.b64encode(img_file.read()).decode()}"
    except FileNotFoundError:
        print(f"Warning: File not found: {image_path}")
        return None  # Return None if the file is missing

# Encode logos to Base64
team_logos = {
    team: image_to_base64(f'./NFL_Data/{team_logo_mapping.get(team, "default.png")}')
    for team in teams
}

# Directory for saving images
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# Create frames for the animation
frames = []
images_list = []
for year in sorted(final_df['Year'].unique()):
    # Include all teams, setting cumulative wins to 0 if no wins in the year
    year_data = final_df[final_df['Year'] <= year].groupby('Winner', observed=False).last().reset_index()
    year_data = year_data.sort_values(by='Cumulative Wins', ascending=False)
    
    # Add logos for the teams in this frame
    images = []
    for _, row in year_data.iterrows():
        logo = team_logos.get(row['Winner'], None)
        if logo:
            images.append(dict(
                source=logo,
                x=row['Cumulative Wins'] + 0.5, 
                y=row['Winner'],
                xref="x",
                yref="y",
                sizex=1.3,
                sizey=1,
                xanchor="right",
                yanchor="middle",
                layer="above"
            ))

    # Save static frame as image
    fig = go.Figure(
        data=[
            go.Bar(
                x=year_data['Cumulative Wins'],
                y=year_data['Winner'],
                orientation='h',
                marker=dict(color=[team_colors[team] for team in year_data['Winner']]),
            )
        ],
        layout=go.Layout(
            title=f"Cumulative Super Bowl Wins (1967-2024)",
            xaxis=dict(title="Cumulative Wins", range=[0, final_df['Cumulative Wins'].max() + 2]),  
            yaxis=dict(title="Teams", categoryorder="total ascending"),
            height=800,
            margin=dict(l=150, r=50, t=100, b=100),  # Adjust bottom margin for buttons
            images=images
        )
    )
    
    # Save the frame as an image file
    image_path = f"{output_dir}/frame_{year}.png"
    fig.write_image(image_path, width=1280, height=720)
    images_list.append(np.array(iio.imread(image_path)))

# Create a video from the saved images
iio.imwrite("cumulative_superbowl_wins.mp4", images_list, fps=3) 

# Cleanup temporary files
for file in os.listdir(output_dir):
    os.remove(os.path.join(output_dir, file))
os.rmdir(output_dir)

print("Video created: cumulative_superbowl_wins.mp4")


[rawvideo @ 0x7f817900aa00] Stream #0: not enough frames to estimate rate; consider increasing probesize


Video created: cumulative_superbowl_wins.mp4
